<a href="https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install -qq git+https://github.com/camenduru/diffusers.git
!pip install -qq transformers ftfy

In [20]:
import torch, os, gc
from diffusers import StableDiffusionPipeline
from torch import autocast

In [5]:
def patch_conv(cls):
	init = cls.__init__
	def __init__(self, *args, **kwargs):
		return init(self, *args, **kwargs, padding_mode='circular')
	cls.__init__ = __init__

patch_conv(torch.nn.Conv2d)

In [6]:
pipe = StableDiffusionPipeline.from_pretrained("/content/gdrive/MyDrive/AI/StableDiffusion/models/stable-diffusion-v1-4-fp16", revision="fp16", torch_dtype=torch.float16).to("cuda")
pipe.safety_checker = lambda images, **kwargs: [images, [False] * len(images)]

batch_idx = 0
outputs_path = "/content/gdrive/MyDrive/AI/StableDiffusion"

In [7]:
from PIL.PngImagePlugin import PngInfo
metadata = PngInfo()

In [22]:
images_path_name = "027"
num_batch_images = 1000
height = 512
width = 512

path = os.path.join(outputs_path, images_path_name)

if os.path.exists(path) == False:
  os.mkdir(path)

In [ ]:
prompt = "cat"
metadata.add_text("Prompt", f"{prompt}")
metadata.add_text("by", "camenduru")

for i in range(num_batch_images):
    gc.collect()
    torch.cuda.empty_cache()
    with autocast("cuda"):
      image = pipe(prompt, height=height, width=width, num_inference_steps=50, eta=0.0, guidance_scale=7.5)["sample"][0]

    image.save(f"{path}/{images_path_name}_{batch_idx}.png", pnginfo=metadata)
    batch_idx += 1